<a href="https://colab.research.google.com/github/ProfessorDong/Deep-Learning-Course-Examples/blob/master/ML_Examples/ClassifyCIFAR10_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

Define hyperparameters and data transformation pipeline

In [2]:
# Define hyperparameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# Define the data transformation pipeline
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

Load the CIFAR-10 image dataset

In [3]:
# Load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# Create data loaders for batch processing
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Define the CNN neural network architecture

In [4]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 128 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

Create an instance of the neural network and define the loss function and optimizer

In [5]:
# Create an instance of the neural network and define the loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

print(net)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


Train the neural network

In [7]:
# Train the neural network
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = net(images)
        # print(images.shape)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/10], Step [100/782], Loss: 1.8844
Epoch [1/10], Step [200/782], Loss: 1.7509
Epoch [1/10], Step [300/782], Loss: 1.5877
Epoch [1/10], Step [400/782], Loss: 1.5527
Epoch [1/10], Step [500/782], Loss: 1.3309
Epoch [1/10], Step [600/782], Loss: 1.6355
Epoch [1/10], Step [700/782], Loss: 1.4911
Epoch [2/10], Step [100/782], Loss: 1.3594
Epoch [2/10], Step [200/782], Loss: 1.0729
Epoch [2/10], Step [300/782], Loss: 1.3262
Epoch [2/10], Step [400/782], Loss: 1.0474
Epoch [2/10], Step [500/782], Loss: 0.9256
Epoch [2/10], Step [600/782], Loss: 1.1332
Epoch [2/10], Step [700/782], Loss: 1.1914
Epoch [3/10], Step [100/782], Loss: 1.0226
Epoch [3/10], Step [200/782], Loss: 0.9879
Epoch [3/10], Step [300/782], Loss: 0.9356
Epoch [3/10], Step [400/782], Loss: 0.9315
Epoch [3/10], Step [500/782], Loss: 0.7401
Epoch [3/10], Step [600/782], Loss: 1.0841
Epoch [3/10], Step [700/782], Loss: 0.9822
Epoch [4/10], Step [100/782], Loss: 0.8510
Epoch [4/10], Step [200/782], Loss: 0.9004
Epoch [4/10

Test the neural network

In [8]:
# Test the neural network
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 73.49 %
